## Deliverable 2. Create a Customer Travel Destinations Map.

In [14]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [15]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("/Users/ali/Documents/Data Camp UOT/WeatherPy/Weather_Database/WeatherPy_Database.csv")
city_data_df.drop("Unnamed: 0", axis =1, inplace= True)
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Narsimhapur,IN,22.9500,79.2000,74.50,92,78,3.53,broken clouds
1,Tiznit,MA,29.5833,-9.5000,67.84,70,5,7.29,clear sky
2,Hofn,IS,64.2539,-15.2082,56.21,83,100,8.93,overcast clouds
3,Punta Arenas,CL,-53.1500,-70.9167,37.51,81,0,23.02,clear sky
4,Champagnole,FR,46.7445,5.9135,61.48,89,100,5.73,overcast clouds


In [16]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 56


In [17]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
selected_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
selected_cities_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
9,Saldanha,ZA,-33.0117,17.9442,51.85,72,100,6.42,overcast clouds
16,Hermanus,ZA,-34.4187,19.2345,52.81,73,100,1.21,overcast clouds
25,Prieska,ZA,-29.6641,22.7474,55.47,39,4,2.75,clear sky
35,Balezino,RU,57.9796,53.0106,53.49,76,93,4.65,overcast clouds
47,Ambatofinandrahana,MG,-20.5500,46.8000,52.23,79,0,3.51,clear sky
49,Ancud,CL,-41.8697,-73.8203,53.42,71,99,9.71,light rain
56,Lebu,CL,-37.6167,-73.6500,51.44,68,7,21.07,clear sky
66,Cape Town,ZA,-33.9258,18.4232,53.11,100,90,5.75,fog
103,Hermanus,ZA,-34.4187,19.2345,52.81,73,100,1.21,overcast clouds
112,Prieska,ZA,-29.6641,22.7474,55.47,39,4,2.75,clear sky


In [18]:
# 4a. Determine if there are any empty rows.
selected_cities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44 entries, 9 to 762
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 44 non-null     object 
 1   Country              43 non-null     object 
 2   Lat                  44 non-null     float64
 3   Lng                  44 non-null     float64
 4   Max Temp             44 non-null     float64
 5   Humidity             44 non-null     int64  
 6   Cloudiness           44 non-null     int64  
 7   Wind Speed           44 non-null     float64
 8   Current Description  44 non-null     object 
dtypes: float64(4), int64(2), object(3)
memory usage: 3.4+ KB


In [19]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
selected_cities_df_nona = selected_cities_df.dropna()
selected_cities_df_nona.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43 entries, 9 to 762
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 43 non-null     object 
 1   Country              43 non-null     object 
 2   Lat                  43 non-null     float64
 3   Lng                  43 non-null     float64
 4   Max Temp             43 non-null     float64
 5   Humidity             43 non-null     int64  
 6   Cloudiness           43 non-null     int64  
 7   Wind Speed           43 non-null     float64
 8   Current Description  43 non-null     object 
dtypes: float64(4), int64(2), object(3)
memory usage: 3.4+ KB


In [20]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = selected_cities_df_nona[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
9,Saldanha,ZA,51.85,overcast clouds,-33.0117,17.9442,
16,Hermanus,ZA,52.81,overcast clouds,-34.4187,19.2345,
25,Prieska,ZA,55.47,clear sky,-29.6641,22.7474,
35,Balezino,RU,53.49,overcast clouds,57.9796,53.0106,
47,Ambatofinandrahana,MG,52.23,clear sky,-20.5500,46.8000,
49,Ancud,CL,53.42,light rain,-41.8697,-73.8203,
56,Lebu,CL,51.44,clear sky,-37.6167,-73.6500,
66,Cape Town,ZA,53.11,fog,-33.9258,18.4232,
103,Hermanus,ZA,52.81,overcast clouds,-34.4187,19.2345,
112,Prieska,ZA,55.47,clear sky,-29.6641,22.7474,


In [21]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        print("Hotel Found")
    except:
        print("Hotel not found. Skipping...")
        pass
        

Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel not found. Skipping...
Hotel Found
Hotel Found
Hotel Found


In [22]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"] != ""]
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
9,Saldanha,ZA,51.85,overcast clouds,-33.0117,17.9442,Blue Bay Lodge
16,Hermanus,ZA,52.81,overcast clouds,-34.4187,19.2345,Misty Waves Boutique Hotel
25,Prieska,ZA,55.47,clear sky,-29.6641,22.7474,Avenue GuestHouse
35,Balezino,RU,53.49,overcast clouds,57.9796,53.0106,Dom Otdykha Lokomotivnykh Brigad
47,Ambatofinandrahana,MG,52.23,clear sky,-20.5500,46.8000,TATAMO
49,Ancud,CL,53.42,light rain,-41.8697,-73.8203,Hotel Arena Gruesa
56,Lebu,CL,51.44,clear sky,-37.6167,-73.6500,Hostal Las Lilas
66,Cape Town,ZA,53.11,fog,-33.9258,18.4232,Southern Sun Waterfront Cape Town
103,Hermanus,ZA,52.81,overcast clouds,-34.4187,19.2345,Misty Waves Boutique Hotel
112,Prieska,ZA,55.47,clear sky,-29.6641,22.7474,Avenue GuestHouse


In [23]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [24]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City </dt><dd>{City}</dd>
<dt>Country </dt><dd>{Country}</dd>
<dt>Current Weather </dt><dd>{Current Description} </dt>and </dt> {Max Temp} </dt> F </dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [28]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(map_type="ROADMAP", zoom_level= 2, center = (28.0000, 20.0000))

# 11b. Display the figure
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))